In [1]:
### IMPORTAMOS CÓDIGO PARA LAS PREGUNTAS DE TRIVIA
import requests
import html
import random
import sys
import time

from utiles.functions import get_session_token, reset_session_token, get_random_category_id, get_trivia_question, objeto_a_cat, new_question

### IMPORTAMOS CÓDIGO PARA LA DETECCIÓN DE IMAGEN
import cv2
import torch
import numpy as np

from utiles.functions import detectar_objeto, detectar_objeto_camara, detectar_objeto_imagen, ingresar_objeto_manual, predicted_object

#### IMPORTAMOS CÓDIGO DEL JUEGO ###

from IPython.display import Image, display
import random

from utiles.functions import linebreak, show_image, initial_settings, play_room, explore_room, examine_item, pet_friendly, start_game, get_next_room_of_door, final_test, end_game

In [5]:
# Definimos la relación de objetos en habitaciones para la búsqueda.

#DOORS:

door_library = {
    "name": "library door",
    "type": "door",
}

door_kitchen = {
    "name": "kitchen door",
    "type": "door",
}

door_living = {
    "name": "living room door",
    "type": "door",
}

door_bedroom = {
    "name": "bedroom door",
    "type": "door",
}

door_outside = {
    "name": "outside door",
    "type": "door",
}


#FURNITURE:


cup = {
    "name": "cup",
    "type": "furniture",
}

bottle = {
    "name": "bottle",
    "type": "furniture",
}

headphones = {
    "name": "headphones",
    "type": "furniture",
}

book = {
    "name": "book",
    "type": "furniture",
}

banana = {
    "name": "banana",
    "type": "furniture",
}

cat = {
    "name": "cat",
    "type": "furniture",
    'category': 'Animals',
}

dog = {
    "name": "dog",
    "type": "furniture",
    'category': 'Animals',
}

basketball = {
    "name": "basketball",
    "type": "furniture",
}

airplane = {
    "name": "airplane",
    "type": "furniture",
}

spacecraft = {
    "name": "Charlie's spacecraft",
    "type": "spacecraft",
}

#ROOMS:

hall = {
    "name": "hall",
    "type": "room",
    "image": 'url',
    "category": "General_Culture",
    "target": [cat,dog],
}

library = {
    "name": "library",
    "type": "room",
    "image": 'url',
    "category": 'Arts_and_Books',
    "target": [book],
    "clue": 'One of those blocks in the shelf would help me. Can you give one to me?',
}

kitchen = {
    "name": "kitchen",
    "type": "room",
    "image": 'url',
    "category": 'Science',
    "target": [banana],
    "clue": "My color is yellow, and I grow on trees, I'm a popular food with apes and monkeys"
}

living_room = {
    "name": "living room",
    "type": "room",
    "image": 'url',
    "category": 'History',
    "target": [airplane],
    "clue": 'What a beautiful model! Is that an Airbus A350? Show it to me, please!'
}

bedroom = {
  "name": "bedroom",
  "type": "room",
  "image": 'url',
  "category": 'Audiovisual',
  "target": [headphones],
  "clue": 'I love listening to music, do you have any item to help me listening to it without bothering anyone else?',
}

outside = {
  "name": "outside",
  "type": "room",
  "image": 'url',
  "category": 'Sports',
  "target": [basketball],
  "clue": 'It is important to do some exercise before a long trip. Wanna try some shots?'
}

space = {
  "name": "outer space",
  "type": "room",
  "image": 'url',
}

#Delimitar habitaciones:

all_rooms = [hall, library, kitchen, living_room, bedroom, outside]

all_doors = [door_library, door_kitchen, door_living, door_bedroom, door_outside]

# define which items/rooms are related

object_relations = {
    ### ROOMS
    "hall": [door_library, door_kitchen, door_living, door_bedroom, door_outside],
    "library": [book,  door_library],
    "kitchen": [banana, door_kitchen],
    "living room": [airplane, door_living],
    "bedroom": [headphones, door_bedroom],
    "outside": [basketball, door_outside, spacecraft],
    ### DOORS
    "library door": [hall, library],
    "kitchen door": [hall, kitchen],
    "living room door": [hall, living_room],
    "bedroom door": [hall, bedroom],
    "outside door": [hall, outside],
}

In [ ]:
game_state = initial_settings()

In [3]:
start_game(game_state)

Hey! Hey!
Are you OK?
Come on, wake up!


I am X4R-L1, but my friends call me Charlie 😊


I need your help, my flying saucer crashed between Orion and Sirius.
I tried hitchhiking, but I stumbled into a black hole and that's how I landed on top on you 😁
I need several objects to repair my ship, could you help me?


Also, I'm a bit thirsty... do you have any bottle to drink?

How will you show me the object?
1. Webcam
2. File image
3. Type manually
bottle manually detected!
That bottle is jut what I needed!
Now get ready for a question about General_Culture!:
Question: In "Fallout 4" which faction is not present in the game?
Options:
1. The Brotherhood of Steel
2. The Enclave
3. The Minutemen
4. The Institute
The Minutemen
I don't think so... 😓
My records say otherwise, the correct answer is The Enclave.
Now, let's go into your house!
You are now in the hall
examine
You examine outside door. You unlock it.
You are now in the outside
It is important to do some exercise before a long trip. Wanna try some shots?

How will you show 